In [7]:
import sys
import torch
import torchvision
import random

import numpy as np
import torch.nn as nn

from tqdm import tqdm
from sklearn import preprocessing
from torch.utils.data import Dataset, DataLoader
from models.models import FeatureSelectNet, Attention

In [8]:
input = torch.ones((32, 200, 10))
model = Attention(
    dim=10,
    num_heads=2,
)
input = input.to('cuda')
model = model.to('cuda')
output = model(input)
print(output.shape)

torch.Size([32, 200, 10])


In [ ]:
dataloader = DataLoader(dataset=cifar10_dataset, # 传入的数据集, 必须参数
                               batch_size=32,       # 输出的batch大小
                               shuffle=True,       # 数据是否打乱
                               num_workers=2)      # 进程数, 0表示只有主进程

In [4]:
def standardize(images):
    # （X-X的均值）/ X的标准差
    images = (images - np.mean(images, axis=0)[None, :, :]) / np.std(images, axis=0)[None, :, :]
    return images

In [20]:
data = np.load('./runs/data/user1/signal.npy')
labels = np.load('./runs/data/user1/labels.npy')
# data = standardize(data)
data.shape, labels.shape


((298383, 8), (298383,))

In [21]:
np.nonzero(np.isnan(data))

(array([], dtype=int64), array([], dtype=int64))

In [34]:
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))
output.shape

torch.Size([5, 3, 20])

In [35]:
input = torch.ones((32, 12, 8))
model = FeatureSelectNet(
    input_shape=input.shape,
    hidden_size=64,
    num_layers=4,
    bias=True,
    batch_first=True,
)
input = input.to('cuda')
model = model.to('cuda')
output = model(input)
print(output.shape)

torch.Size([32, 8])


In [11]:
class MyDataset(Dataset):

    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self,):
        return len(self.x)

In [12]:
n = 100
batch_size = 32
data_size = 100
feature_size = 12
classes = 8
x = torch.rand_like(torch.ones((batch_size*n, data_size, feature_size)))
y = torch.zeros((batch_size*n, classes))

dataloader = torch.utils.data.DataLoader(MyDataset(x,y), shuffle=True, batch_size=batch_size)

In [18]:
model = FeatureSelectNet(
    input_shape=x.shape,
    hidden_size=32,
    num_layers=4,
    bias=True,
)
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
device = 'cuda'

In [29]:

model = model.to(device)

for epoch in range(20):
    model.train()
    acc_loss = torch.zeros(1).to(device)
    acc_num = torch.zeros(1).to(device)
    optimizer.zero_grad()

    sample_num = 0
    dataloader = tqdm(dataloader, file=sys.stdout)
    for step, data in enumerate(dataloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        sample_num += images.shape[0]

        pred = model(images)
        _, idxs_pred = torch.max(pred, dim=1)
        _, idxs_true = torch.max(labels, dim=1)
        acc_num += torch.eq(idxs_pred, idxs_true).sum()

        loss = loss_func(pred, labels)
        loss.backward()

        acc_loss += loss.detach()
        dataloader.desc = '[train epoch {}] loss: {:.3f}, acc: {:.3f}'.format(
                                                                            epoch,
                                                                            acc_loss.item() / (step + 1),
                                                                            acc_num.item() / sample_num
                                                                        )
        optimizer.step()
        optimizer.zero_grad()
    print(acc_loss.item() / (step + 1), acc_num.item() / sample_num)
    

[train epoch 0] loss: 0.000, acc: 0.153: 100%|██████████| 100/100 [00:00<00:00, 255.35it/s]
0.0 0.153125
[train epoch 1] loss: 0.000, acc: 0.142: 100%|██████████| 100/100 [00:00<00:00, 246.30it/s]
0.0 0.1425
[train epoch 2] loss: 0.000, acc: 0.129: 100%|██████████| 100/100 [00:00<00:00, 246.51it/s]
0.0 0.129375
[train epoch 3] loss: 0.000, acc: 0.149: 100%|██████████| 100/100 [00:00<00:00, 245.99it/s]
0.0 0.14875
[train epoch 4] loss: 0.000, acc: 0.140: 100%|██████████| 100/100 [00:00<00:00, 244.90it/s]
0.0 0.1403125
[train epoch 5] loss: 0.000, acc: 0.140: 100%|██████████| 100/100 [00:00<00:00, 242.29it/s]
0.0 0.1396875
[train epoch 6] loss: 0.000, acc: 0.142: 100%|██████████| 100/100 [00:00<00:00, 246.12it/s]
0.0 0.141875
[train epoch 7] loss: 0.000, acc: 0.144: 100%|██████████| 100/100 [00:00<00:00, 243.58it/s]
0.0 0.14375
[train epoch 8] loss: 0.000, acc: 0.148: 100%|██████████| 100/100 [00:00<00:00, 244.76it/s]
0.0 0.1484375
[train epoch 9] loss: 0.000, acc: 0.146: 100%|██████████